In [11]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
import struct
import numpy as np

def float_to_int(f):
    return (struct.unpack('<I', struct.pack('<f', f))[0])

def int_to_float(i):
    return (struct.unpack('<f', struct.pack('<I', i))[0])
    


In [12]:
overlay = Overlay("nn.bit")
fcc1=overlay.forward_fcc_0
fcc1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  y = Register(y=0),
  b = Register(b=0),
  xdimension = Register(xdimension=0),
  ydimension = Register(ydimension=0)
}

In [44]:
BASE_ADDRESS_X=0x4001_0000
BASE_ADDRESS_DX=BASE_ADDRESS_X+xdim*4+4

BASE_ADDRESS_W=BASE_ADDRESS_DX+xdim*4+4

BASE_ADDRESS_B=BASE_ADDRESS_W+xdim*ydim*4+4

BASE_ADDRESS_Y=BASE_ADDRESS_B+ydim*4+4
BASE_ADDRESS_DY=BASE_ADDRESS_Y+ydim*4+4 


mmio_x= MMIO(BASE_ADDRESS_X,xdim*4)
mmio_w= MMIO(BASE_ADDRESS_W,xdim*ydim*4)
mmio_y= MMIO(BASE_ADDRESS_Y,ydim*4)
mmio_b= MMIO(BASE_ADDRESS_B,ydim*4)

OFFSET_X=0x0
OFFSET_Y=0x0
OFFSET_W=0x0
OFFSET_B=0x0

xdim=8
ydim=1

In [45]:
x=[0.053120438009500504, 0.016790520399808884, 0.023369621485471725, 0.031322281807661057, 0.0564102940261364, 0.041052572429180145, 0.04697149619460106, 0.01521462481468916]
w=[0.07526353746652603, 0.03727398440241814, 0.08106990158557892, 0.09472338110208511, 0.06919610500335693, 0.04027724638581276, 0.05966504290699959, 0.01915348507463932]
b=[0.07463822513818741]
y=0
for i in range(8):
    y+=x[i]*w[i]
y+=b[0]
print(y)
for i in range(xdim):
    mmio_x.write(OFFSET_X,float_to_int(x[i]))    
    OFFSET_X=OFFSET_X+4

for i in range(ydim):
    for j in range(xdim):
        mmio_w.write(OFFSET_W,float_to_int(w[i]))
        OFFSET_W += 4

for i in range(ydim):
    mmio_y.write(OFFSET_Y,float_to_int(0))
    mmio_b.write(OFFSET_B,float_to_int(b[i]))
    OFFSET_Y=OFFSET_Y+4
    OFFSET_B=OFFSET_B+4

0.0927744587889836


In [46]:


fcc1.register_map.x=BASE_ADDRESS_X
fcc1.register_map.y=BASE_ADDRESS_Y
fcc1.register_map.w=BASE_ADDRESS_W
fcc1.register_map.b=BASE_ADDRESS_B
fcc1.register_map.xdimension=xdim
fcc1.register_map.ydimension=ydim

In [47]:
fcc1.write(0x00, 1)
fpga_state = fcc1.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = fcc1.read(0x00)
    max_try = max_try -1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        fcc1.write(0x00, 4)
        break

fcc1.write(0x00, 4)
# print(xbuffer)
# print(wbuffer)
# print(ybuffer)
# y[:ydim] = ybuffer[:ydim]

In [48]:
y=[]
OFFSET_X=0x0
OFFSET_Y=0x0
OFFSET_W=0x0
OFFSET_B=0x0
for i in range(ydim):
    y.append(int_to_float(mmio_y.read(OFFSET_Y)))
    OFFSET_Y= OFFSET_Y+4

    
print(y)

[0.0960320234298706]


0
